In [1]:
#looking at the 18 features from >= 4 CIMP+ occurrences
import os
import csv
import statistics

inpdat = "../../../selectedfeatures2.csv"

with open(inpdat,'r') as f:
    it = csv.reader(f)
    listit = list(it)
with open("../../../coadreadDATb4.csv",'r') as f:
    it = csv.reader(f)
    listitorig = list(it)
print("feat # of old:",len(listit[0])-2)

feat # of old: 68


In [2]:
endcol = len(listit[0])-1
endrow = len(listit)

keep = [0]
threeormore = []
mysolution = []
rawsol = "0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0".split(", ")
for x in range(0,len(rawsol)):
    if(rawsol[x] == "1"):
        mysolution.append(x+1)
#the BRAF feat we're looking at has: 16 TP, 4 FP
for y in range(1,endcol):
    TPcount = 0
    FPcount = 0
    for x in range(1,endrow):
        if(float(listit[x][y]) == 1.0):
            if(float(listit[x][endcol]) == 1.0):
                TPcount += 1
            else:
                FPcount += 1
    if (True):
        threeormore.append(y) #list of indices we're looking at
        keep.append(y)

keep.append(len(listit[0])-1)
print(str(len(threeormore)))
threeormore

68


[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68]

In [3]:
results = [["Feature","TP","FP","TN","FN","CIMP Low"]]
model = []
for index in threeormore:
    truepos = trueneg = falsepos = falseneg = 0
    for x in range(1,endrow):
        if(float(listit[x][endcol]) == 1.0):
            if(float(listit[x][index]) == 1.0):
                truepos += 1
            elif(float(listit[x][index]) == 0.0):
                falseneg += 1
        else:
            if(float(listit[x][index]) == 0.0):
                trueneg += 1
            else:
                falsepos += 1
            #else:
            #    print("ERROR:",float(listit[x][index]))
        #else:
        #    print("ERROR:",float(listit[x][endcol]))
    accuracy = 1 - ((falseneg+falsepos)/(trueneg+falseneg+falsepos+truepos))
    newrow = [listit[0][index],truepos,falsepos,trueneg,falseneg,accuracy]
    if(True):
        results.append(newrow)
        model.append(listit[0][index])
    
results

[['Feature', 'TP', 'FP', 'TN', 'FN', 'CIMP Low'],
 ['BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T',
  16,
  4,
  188,
  16,
  0.9107142857142857],
 ['FBXW7_GRCh37_4:153249384-153249384_Missense-Mutation-SNP-C-C-T',
  2,
  1,
  191,
  30,
  0.8616071428571428],
 ['GRB14_GRCh37_2:165365288-165365296_In-Frame-Del-DEL-TTTTTTTTT----',
  7,
  1,
  191,
  25,
  0.8839285714285714],
 ['MBD4_GRCh37_3:129155548-129155557_Frame-Shift-Del-DEL-TTTTTTTTTT----',
  6,
  2,
  190,
  26,
  0.875],
 ['MSH6_GRCh37_2:48030640-48030647_Frame-Shift-Del-DEL-CCCCCCCC----',
  3,
  0,
  192,
  29,
  0.8705357142857143],
 ['PRDM2_GRCh37_1:14108749-14108757_In-Frame-Del-DEL-AAAAAAAAA----',
  5,
  1,
  191,
  27,
  0.875],
 ['HMMR_GRCh37_5:162917426-162917434_In-Frame-Del-DEL-AAAAAAAAA----',
  5,
  1,
  191,
  27,
  0.875],
 ['SEC63_GRCh37_6:108214755-108214764_Frame-Shift-Del-DEL-TTTTTTTTTT----',
  8,
  3,
  189,
  24,
  0.8794642857142857],
 ['KRAS_GRCh37_12:25398284-2

In [4]:
#looking at all features overall
def runGreedyOnTest(model, testdata):
    numrows = len(testdata)
    numcols = len(testdata[0])-1
    results = {}
    for x in range(0, numrows):
        for y in range(0, numcols): #program won't be able to see background/foreground
            if (testdata[x][y] == "1.0"):
                if (testdata[0][y] in model):
                    results[testdata[x][0]] = "1.0"
    for x in range(0,numrows):
        if(testdata[x][0] not in results):
            results[testdata[x][0]] = "-1.0"
    return results
def evalResults(dictofseqs,givendat,notpicked):
    falsepos = falseneg = truepos = trueneg = lowmiss = lowoccur = 0
    for x in range (1,len(givendat)):
        if(givendat[x][0] in dictofseqs):
            if(dictofseqs[givendat[x][0]] == "1.0"):
                if(givendat[x][len(givendat[0])-1] == "-1.0" or givendat[x][len(givendat[0])-1] == "2.0"):
                    falsepos += 1
                #elif(givendat[x][len(givendat[0])-1] == "2.0"):
                #    lowoccur += 1
                else:
                    truepos += 1
                    if givendat[x][0] not in notpicked:
                        notpicked.append(givendat[x][0])
            else:
                if(givendat[x][len(givendat[0])-1] == "1.0"):
                    falseneg += 1
                #elif(givendat[x][len(givendat[0])-1] == "2.0"):
                #    lowmiss += 1
                else:
                    trueneg += 1
    tot = len(dictofseqs)
    overallerror = (falseneg+falsepos)/(trueneg+falseneg+falsepos+truepos)
    sens = truepos/(truepos+falseneg)
    spec = trueneg/(trueneg+falsepos)
    print("False Positives: " + str(falsepos))
    print("False Negatives: " + str(falseneg))
    print("True Positives: " + str(truepos))
    print("True Negatives: " + str(trueneg))
    print("CIMP-Low Occurrences: " + str(lowoccur))
    print("CIMP-Low Not Covered: " + str(lowmiss))
    print("Proportion of false positives: " + str((falsepos/tot)))
    print("Proportion of false negatives:" + str((falseneg/tot)))
    print("Accuracy: " + str((1-overallerror)))
    print("Error rate: " + str(overallerror))
    print("Sensitivity: " + str(sens))
    print("Specificity: " + str(spec))

notpicked = []
dictofseqs = runGreedyOnTest(model,listit)
evalResults(dictofseqs,listit,notpicked)
notpicked

False Positives: 42
False Negatives: 2
True Positives: 30
True Negatives: 150
CIMP-Low Occurrences: 0
CIMP-Low Not Covered: 0
Proportion of false positives: 0.18666666666666668
Proportion of false negatives:0.008888888888888889
Accuracy: 0.8035714285714286
Error rate: 0.19642857142857142
Sensitivity: 0.9375
Specificity: 0.78125


['TCGA-A6-2672',
 'TCGA-A6-2676',
 'TCGA-AA-3516',
 'TCGA-AA-3518',
 'TCGA-AA-3525',
 'TCGA-AA-3543',
 'TCGA-AA-3664',
 'TCGA-AA-3672',
 'TCGA-AA-3710',
 'TCGA-AA-3715',
 'TCGA-AA-3821',
 'TCGA-AA-3833',
 'TCGA-AA-3837',
 'TCGA-AA-3845',
 'TCGA-AA-3870',
 'TCGA-AA-3877',
 'TCGA-AA-3941',
 'TCGA-AA-3947',
 'TCGA-AA-3949',
 'TCGA-AA-3994',
 'TCGA-AA-A00A',
 'TCGA-AA-A00D',
 'TCGA-AA-A00E',
 'TCGA-AA-A00J',
 'TCGA-AA-A01P',
 'TCGA-AA-A022',
 'TCGA-AA-A029',
 'TCGA-AA-A02O',
 'TCGA-AA-A03F',
 'TCGA-AG-3575']

In [5]:

#output stats of features
with open("./statsof68.csv","w") as fun:
    writer = csv.writer(fun)
    writer.writerows(results)

'''
#filter new feats
newdatset = [[row[c] for c in keep] for row in listit]
#output stats of accuracy filter
with open("../../featurefilter/7feats.csv","w") as fun:
    writer = csv.writer(fun)
    writer.writerows(newdatset)
'''

'\n#filter new feats\nnewdatset = [[row[c] for c in keep] for row in listit]\n#output stats of accuracy filter\nwith open("../../featurefilter/7feats.csv","w") as fun:\n    writer = csv.writer(fun)\n    writer.writerows(newdatset)\n'